<a href="https://colab.research.google.com/github/WillAsh/analitica/blob/main/Sentimientos.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q spanish-sentiment-analysis
!pip install -q googletrans==3.1.0a0

In [2]:
import tweepy #API de Twitter
import pandas as pd 
import re #limpieza de texto
import unicodedata #caracteres especiales del idioma 
from classifier import * #modelo de sentimiento pre-entrenado
import datetime 
import nltk #procesamiento de lenguaje natural
from nltk.corpus import stopwords #analisis de palabras irrelevantes
nltk.download('stopwords')
from nltk.sentiment.vader import SentimentIntensityAnalyzer #análisis de sentimiento en inglés
nltk.download('vader_lexicon')
import matplotlib.pyplot as plt

/usr/local/lib/python3.7/dist-packages/sklearn/externals/joblib/__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [3]:
#Claves de cuenta desarrollador
consumer_key = 'SE2j5h44iblufyn0g7nK1yxB7'
consumer_secret = 'AyCZwDYdJlichj9KdOAs7KXWKXv0VtsRUicVdlOoTWu9SKiLHC'
#Claves para comunicarnos con la API
access_token = '1045877872370626561-GeMexqKDTK3TdCjxczBakXyeg0SU7X'
access_token_secret = 'Yk3YNNoeNeVzhVevkSusToKeL8om9jVWEpNonWA5nKma6'

In [4]:
auth = tweepy.OAuthHandler(consumer_key, consumer_secret)
auth.set_access_token(access_token, access_token_secret) 
api = tweepy.API(auth,wait_on_rate_limit=True) 
tweets = [] #crea una lista vacía para almacenar los tweets
hour_per = datetime.timezone(datetime.timedelta(hours=-5))#Configurar la zona horaria 
for tweet in tweepy.Cursor(api.search 
                           ,q = "clases presenciales" #palabra clave
                           ,tweet_mode="extended",
                           lang="es", 
                           ).items(2000):
    
    if tweet.user.followers_count >=400:
        dict_ = {'NOMBRE': tweet.user.screen_name,#Nombre
                'USUARIO': tweet.user.id,#id de usuario
                'NRO_SEGUIDORES': tweet.user.followers_count,  #Nro de seguidores
                'FECHA': tweet.created_at.astimezone(hour_per),#Hora de Creación del tweet
                'TEXTO': tweet.full_text,#Texto
                'LOCALIZACION': tweet.user.location,#Localización
                'NRO_RETWEETS': tweet.retweet_count, #Nro de Retweets que tiene dicho tweet
                'TIPO_TELEFONO': tweet.source #Dispositivo
                }
        tweets.append(dict_)

In [5]:
data=pd.DataFrame(tweets)
data["PERIODO_ID"]=[str(i)[0:10].replace("-","") for i in data["FECHA"]] #Adaptacion de periodo a numérico
data["LINKS"]=[" , ".join(re.findall('http\S+',i)) if len(re.findall('http\S+',i))>0 else "no hay links" for i in data["TEXTO"]]
print(data.shape)#dimensión del dataframe
data.head()

(990, 10)


,NOMBRE,USUARIO,NRO_SEGUIDORES,FECHA,TEXTO,LOCALIZACION,NRO_RETWEETS,TIPO_TELEFONO,PERIODO_ID,LINKS
0,muypuntual,272240496,2243,2021-09-24 09:44:09-05:00,RT @majo_navajasOK: El mismo periodismo que di...,,2,Twitter for Android,20210924,no hay links
1,simon_ulises,1869409050,1808,2021-09-24 09:43:59-05:00,RT @alfodominguez: Quién iba a decir que en el...,"Baja California, México",51,Twitter for Android,20210924,no hay links
2,MiroslAsesora,1074489883,1171,2021-09-24 09:43:52-05:00,RT @OLGALGALVNDGUEZ: @DamarisFany Soy docente ...,"México, Distrito Federal",55,Twitter for Android,20210924,no hay links
3,claudiablopez42,4251818369,3214,2021-09-24 09:43:46-05:00,RT @josedel_rio: El gobierno impulsa clases pr...,,94,Twitter for Android,20210924,no hay links
4,ara_ostertag,1511521862,606,2021-09-24 09:43:25-05:00,Hoy tengo clases presenciales y me quiero morir,,0,Twitter for Android,20210924,no hay links


In [1]:
data=data[data["TEXTO"].str[0:3]!="RT "] #Eliminar Retweets
print(data.shape)
data.head()

NameError: ignored

In [7]:
def limpiar(words): #crea una función para utilizarla posteriormente
    asd = [] #crea una lista nueva
    for word in words: ## recorrer por cada tweet    
        sin_saltos=re.sub('\n', ' ', word.lower()) #eliminar salto de línea y convertir a minúscula
        sin_url=re.sub(r'http\S+', '',sin_saltos)### remover las urls
        sin_nickname=re.sub(r'@\S+', '', sin_url) ### remover los nickname
        sin_hashtag=re.sub(r'#\S+', '', sin_nickname)### remover los hashtag 
        sin_character = re.sub(r'[^\w\s]', '',sin_hashtag) ##remover los signos de puntuacion y/o características especiales
        sin_numero = re.sub("\d+", "", sin_character)## remover los numeros
        sin_tildes = sin_numero.translate(str.maketrans('áéíóúü','aeiouu')) 
        asd.append(sin_tildes.strip()) ##eliminar los espacios en los textos al comienzo y al final y lo almacenamos en una lista   
    return asd

In [8]:
Stopwords=stopwords.words("spanish") 
Stopwords=[unicodedata.normalize("NFKD",str(i)).encode("ascii","ignore").decode("ascii") for i in Stopwords]
def remove_stopwords(news): 
    sin_stopwords=[] 
    for new in news: 
        content=[word for word in new.split() if word not in Stopwords] 
        sin_stopwords.append(" ".join(content)) 
    return sin_stopwords #retornamos lista

In [9]:
data["TEXTO_LIMPIO"]=limpiar(data["TEXTO"]) 
data["clean_stopwords"]=remove_stopwords(data["TEXTO_LIMPIO"])  
print(data.shape)#visualizamos dimension de dataframe
data.head()#visualización

(353, 12)


,NOMBRE,USUARIO,NRO_SEGUIDORES,FECHA,TEXTO,LOCALIZACION,NRO_RETWEETS,TIPO_TELEFONO,PERIODO_ID,LINKS,TEXTO_LIMPIO,clean_stopwords
4,ara_ostertag,1511521862,606,2021-09-24 09:43:25-05:00,Hoy tengo clases presenciales y me quiero morir,,0,Twitter for Android,20210924,no hay links,hoy tengo clases presenciales y me quiero morir,hoy clases presenciales quiero morir
5,MujeresJChacao,973012985357783040,1420,2021-09-24 09:43:14-05:00,#24Septiembre Iniciar las clases presenciales ...,Venezuela,0,Twitter for Android,20210924,no hay links,iniciar las clases presenciales pudiera traer ...,iniciar clases presenciales pudiera traer desa...
6,licona_azul,1389456370722820097,554,2021-09-24 09:42:53-05:00,"Pues dale,no estoy por qué hoy no me toca clas...",,0,Twitter for Android,20210924,https://t.co/tqtAxhaIKb,pues daleno estoy por que hoy no me toca clase...,pues daleno hoy toca clases presencialeshoy vi...
8,Rrodriguez0750,319668065,1151,2021-09-24 09:42:29-05:00,Porque no hay marchas de educadores y estudian...,Panamá. Panamá,0,Twitter for Android,20210924,no hay links,porque no hay marchas de educadores y estudian...,marchas educadores estudiantes universitarios ...
9,PedroACruzSnch1,1193536397254283267,6642,2021-09-24 09:41:07-05:00,Por fin volvemos en la @UMU a las clases plena...,,1,Twitter for iPhone,20210924,no hay links,por fin volvemos en la a las clases plenament...,fin volvemos clases plenamente presenciales mi...


In [10]:
clf = SentimentClassifier()
sentiments=[] #creamos la lista
for i in data["clean_stopwords"]: 
  sentiments.append(clf.predict(i)) #utilizamos el modelo para obtener la polaridad (entre 0-1) siendo lo mas cercano a 0 negativo y lo mas cercano a 1 positivo
sentiments=np.array(sentiments) 
data["ANALISIS_SPN"]=sentiments #creando una nueva columna donde se almacenará la polaridad de cada tweet
data.head() #visualización

,NOMBRE,USUARIO,NRO_SEGUIDORES,FECHA,TEXTO,LOCALIZACION,NRO_RETWEETS,TIPO_TELEFONO,PERIODO_ID,LINKS,TEXTO_LIMPIO,clean_stopwords,ANALISIS_SPN
4,ara_ostertag,1511521862,606,2021-09-24 09:43:25-05:00,Hoy tengo clases presenciales y me quiero morir,,0,Twitter for Android,20210924,no hay links,hoy tengo clases presenciales y me quiero morir,hoy clases presenciales quiero morir,0.079889
5,MujeresJChacao,973012985357783040,1420,2021-09-24 09:43:14-05:00,#24Septiembre Iniciar las clases presenciales ...,Venezuela,0,Twitter for Android,20210924,no hay links,iniciar las clases presenciales pudiera traer ...,iniciar clases presenciales pudiera traer desa...,0.049197
6,licona_azul,1389456370722820097,554,2021-09-24 09:42:53-05:00,"Pues dale,no estoy por qué hoy no me toca clas...",,0,Twitter for Android,20210924,https://t.co/tqtAxhaIKb,pues daleno estoy por que hoy no me toca clase...,pues daleno hoy toca clases presencialeshoy vi...,0.274788
8,Rrodriguez0750,319668065,1151,2021-09-24 09:42:29-05:00,Porque no hay marchas de educadores y estudian...,Panamá. Panamá,0,Twitter for Android,20210924,no hay links,porque no hay marchas de educadores y estudian...,marchas educadores estudiantes universitarios ...,0.019795
9,PedroACruzSnch1,1193536397254283267,6642,2021-09-24 09:41:07-05:00,Por fin volvemos en la @UMU a las clases plena...,,1,Twitter for iPhone,20210924,no hay links,por fin volvemos en la a las clases plenament...,fin volvemos clases plenamente presenciales mi...,0.012249


In [15]:
testing=data.sort_values('ANALISIS_SPN', ascending=False)

In [16]:
testing.head()

,NOMBRE,USUARIO,NRO_SEGUIDORES,FECHA,TEXTO,LOCALIZACION,NRO_RETWEETS,TIPO_TELEFONO,PERIODO_ID,LINKS,TEXTO_LIMPIO,clean_stopwords,ANALISIS_SPN,TEXTO_TRADUCIDO
939,abadiosa,994940589601837056,557,2021-09-24 00:19:21-05:00,Lo único que necesitaba eran clases presencial...,,0,Twitter for iPhone,20210924,no hay links,lo unico que necesitaba eran clases presencial...,unico necesitaba clases presenciales otp,0.847149,lo unico que necesitaba eran clases presencial...
924,LeyvaMurillo,845596440,956,2021-09-24 00:33:12-05:00,"De manera escalonada, para el 11 de octubre el...","Baja California Sur, México",0,Twitter for iPhone,20210924,https://t.co/hk0knPy7aO,de manera escalonada para el de octubre el to...,manera escalonada octubre total estudiantes pl...,0.633699,de manera escalonada para el de octubre el to...
42,OptimusPrime65,132423150,8935,2021-09-24 09:32:24-05:00,@Crazy_8620 @MeybeDom @veritovelazque1 @LILITA...,Me dicen VOS ANDAS EN LA LUNA,0,Twitter for Android,20210924,no hay links,lupitha buenoa dias feliz y productivo viernes...,lupitha buenoa dias feliz productivo viernes u...,0.589510,lupitha good morning happy and productive Frid...
537,juaneszbl,2391432403,1334,2021-09-24 07:01:16-05:00,Encontré una carrera para estudiar y estaba re...,,0,Twitter for Android,20210924,no hay links,encontre una carrera para estudiar y estaba re...,encontre carrera estudiar re entusiasmado acor...,0.542872,encontre una carrera para estudiar y estaba re...
784,CastellanosOK,195954227,3870,2021-09-24 04:25:43-05:00,📚El Gobierno pidió volver a la presencialidad ...,"Rafaela, Argentina",0,Hootsuite Inc.,20210924,"https://t.co/TeBRFB33v8 , https://t.co/0RlDsTORi6",el gobierno pidio volver a la presencialidad p...,gobierno pidio volver presencialidad plena niv...,0.528584,el gobierno pidio volver a la presencialidad p...


In [11]:
from googletrans import Translator
from time import sleep

def traducir(text,**kwargs):
    translator = Translator()
    result = None
    while result == None:
        try:
            result = translator.translate(text,**kwargs)
        except Exception as e:
            print(e)
            translator = Translator()
            sleep(0.5)
            pass
    return result  

In [12]:
lista=data["TEXTO_LIMPIO"].tolist() 
translations = traducir(lista, dest='en') 
traduct=[translation.text for translation in translations] 
data["TEXTO_TRADUCIDO"]=traduct 

In [17]:
sid = SentimentIntensityAnalyzer() 
sentiments=[] 
for sentence in data["TEXTO_TRADUCIDO"]: 
  ss = sid.polarity_scores(sentence) 
  sentiments.append(ss["compound"])  
sentiments=(np.array(sentiments)+1)/2
data["ANALISIS_ENG"]=sentiments 
data.head() 

,NOMBRE,USUARIO,NRO_SEGUIDORES,FECHA,TEXTO,LOCALIZACION,NRO_RETWEETS,TIPO_TELEFONO,PERIODO_ID,LINKS,TEXTO_LIMPIO,clean_stopwords,ANALISIS_SPN,TEXTO_TRADUCIDO,ANALISIS_ENG
4,ara_ostertag,1511521862,606,2021-09-24 09:43:25-05:00,Hoy tengo clases presenciales y me quiero morir,,0,Twitter for Android,20210924,no hay links,hoy tengo clases presenciales y me quiero morir,hoy clases presenciales quiero morir,0.079889,Today I have face-to-face classes and I want t...,0.22130
5,MujeresJChacao,973012985357783040,1420,2021-09-24 09:43:14-05:00,#24Septiembre Iniciar las clases presenciales ...,Venezuela,0,Twitter for Android,20210924,no hay links,iniciar las clases presenciales pudiera traer ...,iniciar clases presenciales pudiera traer desa...,0.049197,starting face-to-face classes could bring a sa...,0.12105
6,licona_azul,1389456370722820097,554,2021-09-24 09:42:53-05:00,"Pues dale,no estoy por qué hoy no me toca clas...",,0,Twitter for Android,20210924,https://t.co/tqtAxhaIKb,pues daleno estoy por que hoy no me toca clase...,pues daleno hoy toca clases presencialeshoy vi...,0.274788,"Well, dale, I'm not because today I don't have...",0.42345
8,Rrodriguez0750,319668065,1151,2021-09-24 09:42:29-05:00,Porque no hay marchas de educadores y estudian...,Panamá. Panamá,0,Twitter for Android,20210924,no hay links,porque no hay marchas de educadores y estudian...,marchas educadores estudiantes universitarios ...,0.019795,because there are no marches of educators and ...,0.29905
9,PedroACruzSnch1,1193536397254283267,6642,2021-09-24 09:41:07-05:00,Por fin volvemos en la @UMU a las clases plena...,,1,Twitter for iPhone,20210924,no hay links,por fin volvemos en la a las clases plenament...,fin volvemos clases plenamente presenciales mi...,0.012249,finally we return to classes fully face-to-fac...,0.38940


In [19]:
sentiments1=data["ANALISIS_SPN"] 
sentiments2=data["ANALISIS_ENG"] 
#sentiments_final=[i if np.abs(i-0.5)>np.abs(j-0.5) else j for i,j in zip(sentiments1,sentiments2)] 
sentiments_final=[((i+j)/2) for i,j in zip(sentiments1,sentiments2) ]
data["ANALISIS_FINAL"]=sentiments_final

In [20]:
etiqueta=np.array(data["ANALISIS_FINAL"])
etiqueta[etiqueta>0.66]=1
etiqueta[etiqueta<0.33]=-1
etiqueta[(etiqueta>=0.33) & (etiqueta<=0.66)]=0
data["POLARIDAD"]=etiqueta
data["sentiments_class"]=data["POLARIDAD"].map({1:"Positivo",0:"Neutral",-1:"Negativo"}) 
data #visualizar

,NOMBRE,USUARIO,NRO_SEGUIDORES,FECHA,TEXTO,LOCALIZACION,NRO_RETWEETS,TIPO_TELEFONO,PERIODO_ID,LINKS,TEXTO_LIMPIO,clean_stopwords,ANALISIS_SPN,TEXTO_TRADUCIDO,ANALISIS_ENG,ANALISIS_FINAL,POLARIDAD,sentiments_class
4,ara_ostertag,1511521862,606,2021-09-24 09:43:25-05:00,Hoy tengo clases presenciales y me quiero morir,,0,Twitter for Android,20210924,no hay links,hoy tengo clases presenciales y me quiero morir,hoy clases presenciales quiero morir,0.079889,Today I have face-to-face classes and I want t...,0.22130,0.150594,-1.0,Negativo
5,MujeresJChacao,973012985357783040,1420,2021-09-24 09:43:14-05:00,#24Septiembre Iniciar las clases presenciales ...,Venezuela,0,Twitter for Android,20210924,no hay links,iniciar las clases presenciales pudiera traer ...,iniciar clases presenciales pudiera traer desa...,0.049197,starting face-to-face classes could bring a sa...,0.12105,0.085124,-1.0,Negativo
6,licona_azul,1389456370722820097,554,2021-09-24 09:42:53-05:00,"Pues dale,no estoy por qué hoy no me toca clas...",,0,Twitter for Android,20210924,https://t.co/tqtAxhaIKb,pues daleno estoy por que hoy no me toca clase...,pues daleno hoy toca clases presencialeshoy vi...,0.274788,"Well, dale, I'm not because today I don't have...",0.42345,0.349119,0.0,Neutral
8,Rrodriguez0750,319668065,1151,2021-09-24 09:42:29-05:00,Porque no hay marchas de educadores y estudian...,Panamá. Panamá,0,Twitter for Android,20210924,no hay links,porque no hay marchas de educadores y estudian...,marchas educadores estudiantes universitarios ...,0.019795,because there are no marches of educators and ...,0.29905,0.159422,-1.0,Negativo
9,PedroACruzSnch1,1193536397254283267,6642,2021-09-24 09:41:07-05:00,Por fin volvemos en la @UMU a las clases plena...,,1,Twitter for iPhone,20210924,no hay links,por fin volvemos en la a las clases plenament...,fin volvemos clases plenamente presenciales mi...,0.012249,finally we return to classes fully face-to-fac...,0.38940,0.200825,-1.0,Negativo
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
970,Osirisljo,1140767473911943169,1317,2021-09-23 23:48:10-05:00,@clinaliciaare Doctora hay que pedir que quite...,,0,Twitter for Android,20210923,no hay links,doctora hay que pedir que quiten las clases pr...,doctora pedir quiten clases presencialesse con...,0.007819,doctora hay que pedir que quiten las clases pr...,0.50000,0.253910,-1.0,Negativo
972,aydcurricular,4843708427,420,2021-09-23 23:47:39-05:00,"Trabajo en equipo, el departamento de Bolívar,...",Colombia,1,Twitter for Android,20210923,"https://t.co/5aQcQaMoZt , https://t.co/eTCXqlyRod",trabajo en equipo el departamento de bolivar l...,trabajo equipo departamento bolivar llego inst...,0.077188,trabajo en equipo el departamento de bolivar l...,0.50000,0.288594,-1.0,Negativo
974,CaraotaDigital,171299971,1888840,2021-09-23 23:46:05-05:00,#Nacionales | Clases presenciales iniciarán la...,Caracas - Venezuela,0,CaraotaDigital,20210923,https://t.co/ZXVWFO3vNC,clases presenciales iniciaran la tercera seman...,clases presenciales iniciaran tercera semana o...,0.046334,clases presenciales iniciaran la tercera seman...,0.50000,0.273167,-1.0,Negativo
976,elsiglocomve,67669448,285682,2021-09-23 23:45:02-05:00,#Regionales | SUMA Aragua rechaza inicio de c...,"Maracay, Venezuela",5,Hootsuite Inc.,20210923,"https://t.co/N8wF5c5vlJ , https://t.co/cgEA3poiBJ",suma aragua rechaza inicio de clases presenciales,suma aragua rechaza inicio clases presenciales,0.084439,suma aragua rechaza inicio de clases presenciales,0.50000,0.292220,-1.0,Negativo


In [21]:

data['FECHA'] = data['FECHA'].apply(lambda a: pd.to_datetime(a).date()) 

In [22]:
xd=[data["FECHA"], data["TEXTO_LIMPIO"], data["sentiments_class"]]

In [23]:
final=pd.DataFrame(xd)

In [24]:
final1=final.transpose()

In [25]:
final1.head()

,FECHA,TEXTO_LIMPIO,sentiments_class
4,2021-09-24,hoy tengo clases presenciales y me quiero morir,Negativo
5,2021-09-24,iniciar las clases presenciales pudiera traer ...,Negativo
6,2021-09-24,pues daleno estoy por que hoy no me toca clase...,Neutral
8,2021-09-24,porque no hay marchas de educadores y estudian...,Negativo
9,2021-09-24,por fin volvemos en la a las clases plenament...,Negativo


In [27]:
final1.to_excel("midopo.xlsx")